In [3]:
from utils.plot_utils import plot_metrics_individually
import os
import glob
from PIL import Image
from collections import defaultdict
import numpy as np
import pandas as pd
from test_utils import calculate_metrics_batch

# === Utility Functions ===
def get_png_paths(folder):
    return sorted([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".png")])

def load_image_pairs(original_paths, pred_paths, resize=False):
    images_a, images_b = [], []
    for p1, p2 in zip(original_paths, pred_paths):
        if os.path.exists(p1) and os.path.exists(p2):
            img_a = Image.open(p1).convert("RGB")
            img_b = Image.open(p2).convert("RGB")
            if resize:
                img_b = img_b.resize((1920, 1080), Image.Resampling.LANCZOS)
            images_a.append(img_a)
            images_b.append(img_b)
        else:
            print(f"⚠️ Missing: {p1} or {p2}")
    return images_a, images_b

def evaluate_video_uni(original_folder, pred_folder, gop=2):
    original_paths = get_png_paths(original_folder)[1::gop]
    pred_paths = get_png_paths(pred_folder)[1::gop]
    print(original_paths[:10], len(original_paths), len(pred_paths))

    orig_imgs, pred_imgs = load_image_pairs(original_paths, pred_paths, resize=True)
    if orig_imgs and pred_imgs:
        metrics = calculate_metrics_batch(orig_imgs, pred_imgs)
        print("✅ Metrics:", metrics)
        return metrics
    print("❌ No valid pairs.")
    return {}

# evaluate_video_uni('data/UVG/Beauty/images','experiments/bi_warp_v3/preds_170k_gop2_q4/Beauty',2)

['data/UVG/Beauty/images/frame_0001.png', 'data/UVG/Beauty/images/frame_0003.png', 'data/UVG/Beauty/images/frame_0005.png', 'data/UVG/Beauty/images/frame_0007.png', 'data/UVG/Beauty/images/frame_0009.png', 'data/UVG/Beauty/images/frame_0011.png', 'data/UVG/Beauty/images/frame_0013.png', 'data/UVG/Beauty/images/frame_0015.png', 'data/UVG/Beauty/images/frame_0017.png', 'data/UVG/Beauty/images/frame_0019.png'] 48 10
✅ Metrics: {'PSNR': 27.872452354431154, 'MS-SSIM': 0.917842036485672, 'LPIPS': 0.11427011787891388, 'FID': 1.0486811399459839, 'FVD': 652912.0873239608}


/home/maryamsana_98/.local/lib/python3.8/site-packages/scipy/linalg/_matfuncs_sqrtm.py:205: RuntimeWarning: invalid value encountered in scalar divide
  arg2 = norm(X.dot(X) - A, 'fro')**2 / norm(A, 'fro')


{'PSNR': 27.872452354431154,
 'MS-SSIM': 0.917842036485672,
 'LPIPS': 0.11427011787891388,
 'FID': 1.0486811399459839,
 'FVD': 652912.0873239608}

In [8]:
def evaluate_all_videos_uni(original_root, pred_root, gop=2, warp=False,mode=1):
    flow_storage = {
        "Beauty": 16525.653333, "Bosphorus": 15291.733333, "HoneyBee": 14057.813333,
        "Jockey": 28380.160000, "ReadySteadyGo": 16112.640000,
        "ShakeNDry": 14059.520000, "YachtRide": 31167.146667
    }

    all_metrics = defaultdict(list)
    for video in sorted(os.listdir(pred_root)):
        orig_path = os.path.join(original_root, video, "images")
        pred_path = os.path.join(pred_root, video)
        if os.path.exists(orig_path) and os.path.exists(pred_path):
            print(f"▶️ Evaluating {video}")
            metrics = evaluate_video_uni(orig_path, pred_path, gop)
            if warp:
                no = 96 - (96 / gop)
                metrics['bpp'] = (flow_storage[video] * no*mode) / (1920 * 1080 * no)
            else:
                metrics['bpp'] = 0
            for k, v in metrics.items():
                all_metrics[k].append(v)
        else:
            print(f"❌ Skipping {video} (missing folders)")

    print("\n📊 Mean Metrics:")
    return {k: np.mean(v) for k, v in all_metrics.items()}

In [10]:
evaluate_all_videos_uni('data/UVG/', 'experiments/bi_warp_v3/preds_170k_gop2_q4' , gop=2, warp=True,mode=2)

▶️ Evaluating Beauty
['data/UVG/Beauty/images/frame_0001.png', 'data/UVG/Beauty/images/frame_0003.png', 'data/UVG/Beauty/images/frame_0005.png', 'data/UVG/Beauty/images/frame_0007.png', 'data/UVG/Beauty/images/frame_0009.png', 'data/UVG/Beauty/images/frame_0011.png', 'data/UVG/Beauty/images/frame_0013.png', 'data/UVG/Beauty/images/frame_0015.png', 'data/UVG/Beauty/images/frame_0017.png', 'data/UVG/Beauty/images/frame_0019.png'] 48 10
✅ Metrics: {'PSNR': 27.872452354431154, 'MS-SSIM': 0.917842036485672, 'LPIPS': 0.11427011787891388, 'FID': 1.0486811399459839, 'FVD': 652912.0873239608}
▶️ Evaluating Bosphorus
['data/UVG/Bosphorus/images/frame_0001.png', 'data/UVG/Bosphorus/images/frame_0003.png', 'data/UVG/Bosphorus/images/frame_0005.png', 'data/UVG/Bosphorus/images/frame_0007.png', 'data/UVG/Bosphorus/images/frame_0009.png', 'data/UVG/Bosphorus/images/frame_0011.png', 'data/UVG/Bosphorus/images/frame_0013.png', 'data/UVG/Bosphorus/images/frame_0015.png', 'data/UVG/Bosphorus/images/frame

{'PSNR': 22.88009853363037,
 'MS-SSIM': 0.8511005452701024,
 'LPIPS': 0.20274109223059247,
 'FID': 2.409281475203378,
 'FVD': 4188828.3597593596,
 'bpp': 0.018683127571924606}

In [16]:
df = pd.read_csv('metrics/bi_warp_inter.csv')

df

,Unnamed: 0,PSNR,MS-SSIM,LPIPS,FID,FVD,bpp,CRF
0,0,22.415098,0.836580,0.219988,2.859836,8.761226e+06,0.018683,1
1,1,22.205788,0.834254,0.213194,2.528050,9.929832e+06,0.018683,4


In [17]:
df.iloc[1] = {'PSNR': 22.88009853363037,
 'MS-SSIM': 0.8511005452701024,
 'LPIPS': 0.20274109223059247,
 'FID': 2.409281475203378,
 'FVD': 4188828.3597593596,
 'bpp': 0.018683127571924606 ,'CRF': 4}

In [18]:
df

,Unnamed: 0,PSNR,MS-SSIM,LPIPS,FID,FVD,bpp,CRF
0,0.0,22.415098,0.836580,0.219988,2.859836,8.761226e+06,0.018683,1
1,NaN,22.880099,0.851101,0.202741,2.409281,4.188828e+06,0.018683,4


In [19]:
df.to_csv('metrics/bi_warp_inter.csv')